In [1]:

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate , train_test_split , StratifiedShuffleSplit
from sklearn import   linear_model, metrics
from sklearn.linear_model import SGDClassifier as SGD

Описание:
Хотим предсказывать выход в просроченный платеж пользователей. Таргет - 1 - вышел, 0 - не вышел

Задачи:

1 - улучшить качество модели от baseline
2 - добавление факторов аргументируйте визуализацией и комментарием в коде
3 - сделать кросс валидацию с перемешиванием для оценки итогового результата StratifiedShuffleSplit, разбиение на 5 фолдов
4 - качество оценивать по roc auc и выведете средний результат по 5 фолдам. Данный результат выведите в коде и запишите в итоговую анкету как ответ к Заданию 1.
6 - напишите 3-5 действий по итогу оценки качества, которые Вы бы сделали для улучшения качества модели, если бы Вам дали на работу еще 5 рабочих дней. Ответ напишите в комментариях к коду в конце.

In [14]:
train_directory = "data/train.csv"
data = pd.read_csv(train_directory, header=0)
data['y'] = [1 if y == 'yes' else 0 for y in data['y']]
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,0
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,0
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,0
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,0
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,0


Для начала посмортим на данные, убедимся, что нет пропусков

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3222 entries, 0 to 3221
Data columns (total 21 columns):
age               3222 non-null int64
job               3222 non-null object
marital           3222 non-null object
education         3222 non-null object
default           3222 non-null object
housing           3222 non-null object
loan              3222 non-null object
contact           3222 non-null object
month             3222 non-null object
day_of_week       3222 non-null object
duration          3222 non-null int64
campaign          3222 non-null int64
pdays             3222 non-null int64
previous          3222 non-null int64
poutcome          3222 non-null object
emp.var.rate      3222 non-null float64
cons.price.idx    3222 non-null float64
cons.conf.idx     3222 non-null float64
euribor3m         3222 non-null float64
nr.employed       3222 non-null float64
y                 3222 non-null int64
dtypes: float64(5), int64(6), object(10)
memory usage: 528.7+ KB


Так как в данных множество числовых переменных, можно начать рассматривать их, не обращая внимания на остальные

In [16]:
data_num = data._get_numeric_data()

In [17]:
data_num.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,487,2,999,0,-1.8,92.893,-46.2,1.313,5099.1,0
1,39,346,4,999,0,1.1,93.994,-36.4,4.855,5191.0,0
2,25,227,1,999,0,1.4,94.465,-41.8,4.962,5228.1,0
3,38,17,3,999,0,1.4,94.465,-41.8,4.959,5228.1,0
4,47,58,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,0


Посмотрим на уникальные значения по переменным

In [43]:
data_num.previous.unique()

array([0, 2, 1, 3, 5, 4, 6], dtype=int64)

In [48]:
data_num['nr.employed'].unique()

array([5099.1, 5191. , 5228.1, 5195.8, 4963.6, 5008.7, 5076.2, 4991.6,
       5017.5, 5023.5, 5176.3])

## Случайный лес на всех числовых переменных ##

Так как случайный лес не чувствителен к масштабированию, можно использовать его без применения дополнительных модификаций в данных

In [48]:
X = data_num.drop(columns='y',axis=1)
y = data_num.y

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)

In [33]:
rfc = RandomForestClassifier(n_estimators=500)
rfc.fit(X_train,y_train)
y_pred = rfc.predict_proba(X_test)
#rfc.score(X_test,y_test)
metrics.roc_auc_score(y_test, y_pred[:,1])

0.9149463550148482

Немного улучшили baseline

roc_auc считается по функции predict_proba для того, чтобы учитывать не просто бинарные ответы, но и  вероятности классов, от чего значение этой метрики будет выше, чем при использовании бнарных лейблов (predict)

## Логистическая регрессия с нормализацией на числовых переменных ##

Линейные модели чувствительны к масштабированию, поэтому перед их использованием применим StandardScaler

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [23]:
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)


In [28]:
lr = LogisticRegression()
lr.fit(X_scaled_train,y_train)
y_pred_lr = lr.predict_proba(X_scaled_test)
metrics.roc_auc_score(y_test, y_pred_lr[:,1])

0.9000862151547082

Тут аналогично RF

## Catboost на всей выборке с автоматической обработкой категориальных переменных##

In [34]:
from catboost import CatBoostClassifier

In [35]:
X = data.drop(columns = 'y',axis=1)
y = data.y

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)

In [37]:
cat_features = data.select_dtypes(include=['object'])

In [39]:
cb = CatBoostClassifier(cat_features = cat_features)
cb.fit(X_train,y_train,verbose=-1)
y_pred = cb.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_pred[:,1])

Learning rate set to 0.021901


0.9231727176932656

Тут аналогично двум предыдущим

## По кросс-валидируем немного на логистической регрессии ##

In [61]:
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score

По условиям задания необходимо использовать StratifiedShuffleSplit

In [41]:
sss = StratifiedShuffleSplit(n_splits=5,random_state=42)

Снова переопределим наши переменные и отмасштабируем их после CatBoost-а

In [52]:
X = data_num.drop(columns='y',axis=1)
y = data_num.y

In [53]:
scale = StandardScaler()
X = scale.fit_transform(X)

In [64]:
scores = cross_val_score(lr, X, y, cv=sss, scoring='roc_auc')
print(scores)

[0.90030972 0.92218351 0.91144019 0.93253968 0.95625242]


Среднее значение roc_auc для логистической регрессии по функции predict_proba

In [65]:
scores.mean()

0.9245451025938832

## Если бы я попал на работу и мне дали еще немного времени на эту задачу ##

1. Прежде всего, я бы проанализировал текстовые переменные и преминил бы к ним label encoding для бинарных и one hot encoding для множественных.
2. Посмотрел бы на значимость переменных, возможно, избавился бы от некоторых незначащих. Так как сам набор данных маленький, слишком много измерений ему ни к чему.  
3. Поперебирал бы параметры на модели градиентного бустинга с кросс-валидацией по 5 фолдам при помощи hyperopt или grid_search.
4. После всего, оптимизировал бы код и добавил больше комментариев к нему.

